# Siniestros Viales en la Ciudad Autónoma de Buenos Aires
### (Lesiones no fatales)

Con base en los datos alojados en la página de "Buenos Aires Data" se extraen los datasets de "homicidios" en Excel que contienen las víctimas mortales, lugar de los hechos y los actores involucrados.

Extracción, Transformación y Limpiza de los datos:

#### 1. Descarga de librería necesarias:

In [1]:
import pandas as pd                                                           # Para abrir y transformar los archivos
from urllib.request import urlretrieve                                        # Para leer y guardar los archivos originales
import openpyxl                                                               # Para manipular los archivos en excel
from pyproj import Proj, transform                                            # Para transformar coordenadas
import numpy as np                                                            # Para tratar nulos y realizar operaciones matemáticas
import warnings
warnings.filterwarnings("ignore")                                             # Ignorar todas las advertencias

#### 2. Descargar archivo de "homicidios" y guardarlo en la carpeta "Datos_Originales":

In [2]:
# URL del archivo Excel
url_lesiones = 'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/victimas-siniestros-viales/lesiones.xlsx'

# Ruta local donde se guardará el archivo
Ubicación_lesiones = "Datos_Originales/Lesiones_CABA.xlsx"

# Descargar el archivo desde la URL y guardarlo localmente
urlretrieve(url_lesiones, Ubicación_lesiones)

# Crear un objeto ExcelFile para manejar la estructura del archivo Excel
wd_lesiones = openpyxl.load_workbook(Ubicación_lesiones, data_only=True)

# Obtener la lista de nombres de hojas
sheet_names_lesiones = wd_lesiones.sheetnames

# Imprimir los nombres de las hojas disponibles
print("Hojas disponibles:", sheet_names_lesiones)

Hojas disponibles: ['HECHOS', 'DICCIONARIO_HECHOS', 'VICTIMAS', 'DICCIONARIO_VICTIMAS']


A través de openpyxl, se crea un objeto "ExcelFile" del que se pueden obtener los nombres con el método "Sheetnames" y manipular eficientemente el archivo de excel.

#### 3. Crear Datafrome de la hoja de "Hechos" del archivo de excel "Lesiones":

In [3]:
# Crear un objeto ExcelFile para manejar el archivo Excel
xl = pd.ExcelFile(Ubicación_lesiones)

# crear dataframe
df_lesiones = xl.parse('HECHOS',skiprows=0)

# Reiniciar los índices después de la eliminación
df_lesiones.reset_index(drop=True, inplace=True)

#### 3.1. Revisar la calidad de la información por columnas:

In [4]:
df_lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23785 non-null  object 
 7   franja_hora            23780 non-null  object 
 8   direccion_normalizada  23732 non-null  object 
 9   comuna                 23616 non-null  object 
 10  tipo_calle             23785 non-null  object 
 11  otra_direccion         23785 non-null  object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

Aparentemente solo los campos "Cruce" y "Altura" tienen registros nulos, pero se debe revisar más específicamente el contenido de las demás columnas.

A continuación se revisan los datos nulos de las columnas que se consideran relevantes:

In [5]:
print('Victima:',df_lesiones['victima'].unique())  # Normalizar los datos en función del archivo de Homicidios
print('--------------------------------------------')   # Normalizar los datos en función del archivo de Homicidios
print('Acusado:',df_lesiones['acusado'].unique())
print('--------------------------------------------')
print('Comuna:',df_lesiones['comuna'].unique()) # Cambiar "No especificada" y "NaN" por "SD"
print('--------------------------------------------')
print('Longitud:',(df_lesiones['longitud'] == "SD").sum()) # veirificar si se pueden imputar datos con comuna y Geo_CABA.
print('Latitud:',(df_lesiones['latutid'] == "SD").sum())
print('--------------------------------------------')
print('Participantes:',df_lesiones['participantes'].unique()) # demasiadas Variacioanes, eliminar

Victima: ['CICLISTA' 'AUTO' 'SD' 'PEATON' 'MOTO' 'TRANSPORTE PUBLICO' 'CAMIONETA'
 'MOVIL' 'OTRO' 'CAMION' 'MONOPATIN' 'UTILITARIO' 'TAXI' 'MIXTO']
--------------------------------------------
Acusado: ['SD' 'AUTO' 'CICLISTA' 'MOVIL' 'CAMIONETA' 'MOTO' 'TAXI'
 'TRANSPORTE PUBLICO' 'CAMION' 'OBJETO FIJO' 'OTRO' 'UTILITARIO' 'PEATON'
 'MONOPATIN']
--------------------------------------------
Comuna: [14 8 7 3 15 11 9 1 4 12 2 5 6 13 10 'No Especificada' nan 'SD']
--------------------------------------------
Longitud: 1209
Latitud: 1209
--------------------------------------------
Participantes: ['CICLISTA-SD' 'AUTO-SD' 'SD-SD' 'PEATON-SD' 'MOTO-SD'
 'TRANSPORTE PUBLICO-SD' 'CAMIONETA-SD' 'MOVIL-SD' 'OTRO-SD' 'CAMION-SD'
 'MONOPATIN-SD' 'UTILITARIO-SD' 'TAXI-SD' 'CICLISTA-AUTO' 'MOTO-CICLISTA'
 'CICLISTA-MOVIL' 'PEATON-CICLISTA' 'CICLISTA-CAMIONETA' 'CICLISTA-MOTO'
 'CICLISTA-TAXI' 'CICLISTA-TRANSPORTE PUBLICO' 'CICLISTA-CAMION'
 'CICLISTA-OBJETO FIJO' 'CICLISTA-OTRO' 'CICLISTA-CICLISTA'


Podemos observar que para la columna de "Victima" se tiene valores que nombrados de diferente fomra que el dataset de Homicidios y deben normalizarse. También que aprentemente la mayoría de las columnas no tenían datos nulos, pero si "SD" (sin dato) y que por lo general son una cantidad significativa. Así que para poder usar este dataset, es conveniente tratar de imputar al máximo los registros "SD". En caso de la gravedad del incidente, el artículo "NOTAS_LESIONES_SINIESTRO_VIAL" extraído de la misma funte de los datos, indica que "SD" para este campo es en mayor probabilidad "LEVE". Para el caso de las coordenadas, se utlizarán la georeferenciación plana para imputar algunos nulos (SD) y en su defecto la ubicación de la comuna.

#### 3.1.1. Revisar columna de "COMUNA", cambiar datos nulos y "No especificada" por "SD":

In [6]:
# Imputar "SD" en lugar de "0" en la columna 'comuna'
df_lesiones.loc[df_lesiones['comuna'] == 'No Especificada', 'comuna'] = 'SD'
df_lesiones.loc[df_lesiones['comuna'].isna(), 'comuna'] = 'SD'
print('Comuna:',df_lesiones['comuna'].unique())

Comuna: [14 8 7 3 15 11 9 1 4 12 2 5 6 13 10 'SD']


#### 3.1.2. Normalizar datos de "Victima" y "Acusado" a los valores del dataset de "Homicidios":

Columna "Victima":

In [7]:
df_lesiones.loc[df_lesiones['victima'] == 'CICLISTA', 'victima'] = 'BICICLETA'
df_lesiones.loc[df_lesiones['victima'] == 'TRANSPORTE PUBLICO', 'victima'] = 'PASAJEROS'
df_lesiones.loc[df_lesiones['victima'] == 'CAMIONETA', 'victima'] = 'AUTO'
df_lesiones.loc[df_lesiones['victima'] == 'CAMION', 'victima'] = 'CARGAS'
df_lesiones.loc[df_lesiones['victima'] == 'MONOPATIN', 'victima'] = 'OTRO'
df_lesiones.loc[df_lesiones['victima'] == 'UTILITARIO', 'victima'] = 'AUTO'
df_lesiones.loc[df_lesiones['victima'] == 'TAXI', 'victima'] = 'PASAJEROS'
df_lesiones.loc[df_lesiones['victima'] == 'MIXTO', 'victima'] = 'MULTIPLE'

Columna "Acusado":

In [8]:
df_lesiones.loc[df_lesiones['acusado'] == 'CICLISTA', 'acusado'] = 'BICICLETA'
df_lesiones.loc[df_lesiones['acusado'] == 'TRANSPORTE PUBLICO', 'acusado'] = 'PASAJEROS'
df_lesiones.loc[df_lesiones['acusado'] == 'CAMIONETA', 'acusado'] = 'AUTO'
df_lesiones.loc[df_lesiones['acusado'] == 'CAMION', 'acusado'] = 'CARGAS'
df_lesiones.loc[df_lesiones['acusado'] == 'MONOPATIN', 'acusado'] = 'OTRO'
df_lesiones.loc[df_lesiones['acusado'] == 'UTILITARIO', 'acusado'] = 'AUTO'
df_lesiones.loc[df_lesiones['acusado'] == 'TAXI', 'acusado'] = 'PASAJEROS'
df_lesiones.loc[df_lesiones['acusado'] == 'MIXTO', 'acusado'] = 'MULTIPLE'

#### 3.1.3. Imputar datos de Coordenadas:

Crear función que busca las transforma las coordenada planas de CABA a Latitud y Longitud:

ver: https://epsg.io/transform#s_srs=9498&t_srs=4326&x=104991.4633392&y=107720.9664748

In [16]:
# Define la proyección original (EPSG:5343) con transformación 9264
proyeccion_origen = Proj(init='epsg:9498 ', preserve_units=False)

# Define la proyección de destino (WGS84 para latitud y longitud)
proyeccion_destino = Proj(init='epsg:4326')

# Función para convertir coordenadas planas a latitud y longitud
def convertir_a_latitud_longitud(x, y):
    try:
        latitud, longitud = transform(proyeccion_origen, proyeccion_destino, x, y)
        return latitud, longitud
    except Exception as e:
        print(f"Error: {e}")
    return None, None

Usar fórmula para convertir coordenadas planas CABA en coordenadas mundiales:

In [17]:

# Extraer las coordenadas X y Y de la columna 'geocodificacion_CABA'
df_lesiones[['X', 'Y']] = df_lesiones['geocodificacion_CABA'].str.extract(r'Point \((.*?) (.*?)\)').astype(float)

# Aplicar la función a las columnas de coordenadas X y Y, y crear nuevas columnas para latitud y longitud
df_lesiones[['Latitud', 'Longitud']] = df_lesiones.apply(lambda row: convertir_a_latitud_longitud(row['X'], row['Y']), axis=1, result_type='expand')

# Mostrar el DataFrame actualizado
df_lesiones

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,moto,auto,transporte_publico,camion,ciclista,gravedad,X,Y,Latitud,Longitud
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,SD,SD,SD,SD,x,SD,104991.463339,107720.966475,-57.540249,-34.285755
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,SD,x,SD,SD,SD,SD,101813.847125,95578.555072,-57.573601,-34.395458
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,SD,SD,SD,SD,SD,SD,99538.586603,94643.418718,-57.598255,-34.404065
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2,SD,7,...,x,SD,SD,SD,SD,SD,102372.168310,97994.137503,-57.567762,-34.373641
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4,SD,3,...,SD,SD,x,SD,SD,SD,105968.982868,102737.173469,-57.529137,-34.330596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,1,2021,12,31,2021-12-31 00:00:00,19:30:00,19,"ALBERDI, JUAN BAUTISTA AV. y GUAMINI",9,...,SD,SD,SD,SD,SD,SD,95400.544191,96621.600771,-57.643430,-34.386544
23781,LC-2021-0652865,2,2021,12,31,2021-12-31 00:00:00,19:40:00,19,"ALBERDI, JUAN BAUTISTA AV. 4436",9,...,SD,SD,SD,SD,SD,SD,97705.449147,98614.882547,-57.618550,-34.368406
23782,LC-2021-0652907,1,2021,12,31,2021-12-31 00:00:00,20:00:00,20,SD,1,...,SD,SD,SD,SD,SD,SD,107377.106949,105120.787397,-57.514079,-34.308994
23783,LC-2021-0652921,1,2021,12,31,2021-12-31 00:00:00,22:00:00,22,LINIERS VIRREY y MORENO,5,...,x,0,x,0,0,GRAVE,104472.702046,101660.930364,-57.545288,-34.340420


In [22]:
df_lesiones[['Latitud','Longitud','latutid','longitud']].head(10)

,Latitud,Longitud,latutid,longitud
0,-57.540249,-34.285755,-34.559658,-58.408911
1,-57.573601,-34.395458,-34.669125,-58.443510
2,-57.598255,-34.404065,-34.677556,-58.468335
3,-57.567762,-34.373641,-34.647349,-58.437425
4,-57.529137,-34.330596,-34.604579,-58.398225
5,-57.570334,-34.314418,-34.588108,-58.439392
6,-57.649778,-34.316156,-34.589282,-58.519116
7,-57.658414,-34.377093,-34.650156,-58.528413
8,-57.502554,-34.324245,-34.598416,-58.371488
9,-57.535185,-34.374447,-34.648387,-58.404748


Se observan diferencias en la conversión debido a que no se conocía con la precisión ajustaremos con el error medio:

In [45]:
# Filtra los registros con valores menores a -30 en la columna 'Latitud'
df_filtrado = df_lesiones[(df_lesiones['Latitud'] < -38) & (df_lesiones['Latitud'] > -60) & (df_lesiones['latutid'] > -38)]


# Calcula los promedios excluyendo los valores nulos
promedio_latitud = df_filtrado['Latitud'].mean(skipna=True)
promedio_longitud = df_filtrado['Longitud'].mean(skipna=True)
promedio_latutid = df_filtrado['latutid'].mean(skipna=True)
promedio_longitud_2 = df_filtrado['longitud'].mean(skipna=True)

# Calcula las diferencias promedio
dif_Lat = promedio_latitud - promedio_longitud_2
dif_Lon = promedio_longitud- promedio_latutid

print(f"Diferencia entre promedio de Latitud y promedio de Longitud: {dif_Lat}")
print(f"Diferencia entre promedio de Longitud y promedio de latutid: {dif_Lon}")

Diferencia entre promedio de Latitud y promedio de Longitud: 0.869315711317455
Diferencia entre promedio de Longitud y promedio de latutid: 0.2736702407703788


Conociendo el error medio corregimos los datos convertidos de coordenadas:

In [46]:
df_lesiones['Latitud'] = df_lesiones['Latitud'] - dif_Lat
df_lesiones['Longitud'] = df_lesiones['Longitud'] - dif_Lon

In [47]:
df_lesiones[['Latitud','Longitud','latutid','longitud']].head(5)

,Latitud,Longitud,latutid,longitud
0,-58.409565,-34.559425,-34.559658,-58.408911
1,-58.442917,-34.669128,-34.669125,-58.443510
2,-58.467571,-34.677735,-34.677556,-58.468335
3,-58.437078,-34.647311,-34.647349,-58.437425
4,-58.398452,-34.604267,-34.604579,-58.398225


Verificamos que los registros se han corregido.

Imputar los datos de coordenadas con los datos extraídos de las coordenadas planas:

In [48]:
# Imputar los datos vacíos y aquellos con "SD" en "latutid" con los valores de "Longitud"
df_lesiones['latutid'].fillna(df_lesiones['Longitud'], inplace=True)
df_lesiones['latutid'] = np.where(df_lesiones['latutid'] == 'SD', df_lesiones['Longitud'], df_lesiones['latutid'])

# Imputar los datos vacíos y aquellos con "SD" en "longitud" con los valores de "Latitud"
df_lesiones['longitud'].fillna(df_lesiones['Latitud'], inplace=True)
df_lesiones['longitud'] = np.where(df_lesiones['Longitud'] == 'SD', df_lesiones['Latitud'], df_lesiones['longitud'])

# Renombrar 'latutid' con 'latitud'
df_lesiones.rename(columns={'latutid': 'latitud'}, inplace=True)

# Mostrar el DataFrame actualizado
df_lesiones.head()

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,moto,auto,transporte_publico,camion,ciclista,gravedad,X,Y,Latitud,Longitud
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9,SD,14,...,SD,SD,SD,SD,x,SD,104991.463339,107720.966475,-58.409565,-34.559425
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1,SD,8,...,SD,x,SD,SD,SD,SD,101813.847125,95578.555072,-58.442917,-34.669128
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2,SD,8,...,SD,SD,SD,SD,SD,SD,99538.586603,94643.418718,-58.467571,-34.677735
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2,SD,7,...,x,SD,SD,SD,SD,SD,102372.168310,97994.137503,-58.437078,-34.647311
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4,SD,3,...,SD,SD,x,SD,SD,SD,105968.982868,102737.173469,-58.398452,-34.604267


### 3.2. Elimnar las columnas innecesarias:

In [49]:
df_lesiones.columns

Index(['id', 'n_victimas', 'aaaa', 'mm', 'dd', 'fecha', 'hora', 'franja_hora',
       'direccion_normalizada', 'comuna', 'tipo_calle', 'otra_direccion',
       'calle', 'altura', 'cruce', 'geocodificacion_CABA', 'longitud',
       'latitud', 'victima', 'acusado', 'participantes', 'moto', 'auto',
       'transporte_publico', 'camion', 'ciclista', 'gravedad', 'X', 'Y',
       'Latitud', 'Longitud'],
      dtype='object')

In [50]:
# Lista de columnas a eliminar
columnas_lec_eliminar = ['n_victimas', 'aaaa', 'mm', 'dd', 'franja_hora',
       'direccion_normalizada', 'otra_direccion',
       'calle', 'altura', 'cruce', 'geocodificacion_CABA', 'participantes', 'moto', 'auto',
       'transporte_publico', 'camion', 'ciclista', 'gravedad', 'X', 'Y',
       'Latitud', 'Longitud']

# Eliminar las columnas especificadas
df_lesiones = df_lesiones.drop(columns=columnas_lec_eliminar)

# Formato de fecha
df_lesiones['fecha'] = pd.to_datetime(df_lesiones['fecha'], errors='coerce')

# Mostrar el DataFrame actualizado
df_lesiones

,id,fecha,hora,comuna,tipo_calle,longitud,latitud,victima,acusado
0,LC-2019-0000179,2019-01-01,09:00:00,14,SD,-5.840891e+01,-3.455966e+01,BICICLETA,SD
1,LC-2019-0000053,2019-01-01,01:55:00,8,SD,-5.844351e+01,-3.466913e+01,AUTO,SD
2,LC-2019-0000063,2019-01-01,02:00:00,8,SD,-5.846834e+01,-3.467756e+01,SD,SD
3,LC-2019-0000079,2019-01-01,02:30:00,7,SD,-5.843742e+01,-3.464735e+01,PEATON,SD
4,LC-2019-0000082,2019-01-01,04:30:00,3,SD,-5.839822e+01,-3.460458e+01,AUTO,SD
...,...,...,...,...,...,...,...,...,...
23780,LC-2021-0652849,2021-12-31,19:30:00,9,AVENIDA,-5.851348e+01,-3.465971e+01,SD,SD
23781,LC-2021-0652865,2021-12-31,19:40:00,9,AVENIDA,-5.848833e+01,-3.464175e+01,SD,SD
23782,LC-2021-0652907,2021-12-31,20:00:00,1,SD,-5.838289e+01,-3.458308e+01,SD,SD
23783,LC-2021-0652921,2021-12-31,22:00:00,5,CALLE,-5.841453e+01,-3.461429e+01,MOTO,PASAJEROS


Se concidera liasta esta tabla para trabajaren el EDA.

In [52]:
df_lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          23785 non-null  object        
 1   fecha       23785 non-null  datetime64[ns]
 2   hora        23785 non-null  object        
 3   comuna      23785 non-null  object        
 4   tipo_calle  23785 non-null  object        
 5   longitud    23785 non-null  float64       
 6   latitud     23785 non-null  float64       
 7   victima     23785 non-null  object        
 8   acusado     23785 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 1.6+ MB


### 3.3. Exportar datos en CSV a la carpeta "Datos_Limpios"

In [53]:
df_lesiones.to_csv('Datos_limpios/Hechos_Lesiones_CABA.csv',index=False)

#### 4. Crear Datafrome de la hoja de "Victimas" del archivo de excel "Lesiones":

In [54]:
# Crear un objeto ExcelFile para manejar el archivo Excel
xl = pd.ExcelFile(Ubicación_lesiones)

# crear dataframe
df_Lesionados = xl.parse('VICTIMAS',skiprows=0)

# Reiniciar los índices después de la eliminación
df_Lesionados.reset_index(drop=True, inplace=True)

#### 4.1. Verificamos la calidad de los datos

In [55]:
df_Lesionados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  27605 non-null  object        
 6   SEXO              27605 non-null  object        
 7   EDAD_VICTIMA      27605 non-null  object        
 8   GRAVEDAD          27605 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.9+ MB


Aparente mente no se encuentran valores nulos sin embargo se precede a revisar si existe registros como "SD" como en el dataset anterior:

In [56]:
print('Victima:',df_Lesionados['VEHICULO_VICTIMA'].unique())  # Se debe normalizar los datos en función del archivo de Homicidios
print('--------------------------------------------')   
print('sexo:',df_Lesionados['SEXO'].unique())   # Se debe transformar valores en mayúsculas
print('--------------------------------------------')
print('Edad:',df_Lesionados['EDAD_VICTIMA'].unique())  # Se deb transformar valores en mayúsculas
print('--------------------------------------------')
print('Gravedad:',df_Lesionados['GRAVEDAD'].unique())  # Se deb transformar valores en mayúsculas

Victima: ['sd' 'MOTO' 'AUTO' 'TAXI' 'PEATON' 'CICLISTA' 'SD' 'TRANSPORTE PUBLICO'
 'MOVIL' 'CAMION' 'UTILITARIO' 'MONOPATIN' 'MIXTO' 'OTRO']
--------------------------------------------
sexo: ['Varon' 'SD' 'Mujer' 'varon' 'mujer' 'MUJER' 'sd' 'Mujer ']
--------------------------------------------
Edad: [57 'SD' 45 27 32 21 37 33 25 23 42 53 1 62 67 75 40 48 3 49 '25' 34 52 44
 28 59 50 73 '26' 51 16 43 38 64 36 84 22 26 '32' 20 5 39 56 69 55 46 29
 41 54 74 70 6 30 12 83 24 35 68 15 31 19 '30' 9 2 13 18 17 58 60 47 '20'
 7 63 66 80 65 71 79 85 61 77 90 '43' '58' '45' 76 '54' '55' 89 '74' '19'
 78 91 97 10 72 88 11 81 '31' 8 14 '50' '21' 96 '27' '34' '40' '28' '59'
 '39' 82 87 86 '38' '37' 'sd' '44' '29' '35' '42' '18' 4 '23' '41' 92 '46'
 '72' '36' '24' '87' '16' '14' '33' '61' '12' '49' '62' '90' '22' '60'
 '64' '56' '78' '65' '51' '53' '66' '71' '76' '47' '52' '48' '13' '63'
 '68' '15' '57' '70' '73' '80' '67' '5' '84' '94' '86' '3' '69' '6' '17'
 '91' '7' '8' '82' '75' '85' '77' '4'

#### 4.2. Realizar transformaciones de valores en minúscula y "SD" a "leve" en la columna gravedad:

In [57]:
# Cambiar a mayúsculas
df_Lesionados['VEHICULO_VICTIMA'] = df_Lesionados['VEHICULO_VICTIMA'].str.upper()
df_Lesionados['SEXO'] = df_Lesionados['SEXO'].str.upper()
df_Lesionados['GRAVEDAD'] = df_Lesionados['GRAVEDAD'].str.upper()

# Reemplazar "SD" por "LEVE" en la columna "GRAVEDAD"
df_Lesionados['GRAVEDAD'] = df_Lesionados['GRAVEDAD'].replace("SD", "LEVE") # Esto basado en la consideración escrita en "NOTAS_HOMICIDIOS..."

print('Victima:',df_Lesionados['VEHICULO_VICTIMA'].unique()) 
print('--------------------------------------------')   
print('sexo:',df_Lesionados['SEXO'].unique())  
print('--------------------------------------------')
print('Edad:',df_Lesionados['EDAD_VICTIMA'].unique()) 
print('--------------------------------------------')
print('Gravedad:',df_Lesionados['GRAVEDAD'].unique())


Victima: ['SD' 'MOTO' 'AUTO' 'TAXI' 'PEATON' 'CICLISTA' 'TRANSPORTE PUBLICO'
 'MOVIL' 'CAMION' 'UTILITARIO' 'MONOPATIN' 'MIXTO' 'OTRO']
--------------------------------------------
sexo: ['VARON' 'SD' 'MUJER' 'MUJER ']
--------------------------------------------
Edad: [57 'SD' 45 27 32 21 37 33 25 23 42 53 1 62 67 75 40 48 3 49 '25' 34 52 44
 28 59 50 73 '26' 51 16 43 38 64 36 84 22 26 '32' 20 5 39 56 69 55 46 29
 41 54 74 70 6 30 12 83 24 35 68 15 31 19 '30' 9 2 13 18 17 58 60 47 '20'
 7 63 66 80 65 71 79 85 61 77 90 '43' '58' '45' 76 '54' '55' 89 '74' '19'
 78 91 97 10 72 88 11 81 '31' 8 14 '50' '21' 96 '27' '34' '40' '28' '59'
 '39' 82 87 86 '38' '37' 'sd' '44' '29' '35' '42' '18' 4 '23' '41' 92 '46'
 '72' '36' '24' '87' '16' '14' '33' '61' '12' '49' '62' '90' '22' '60'
 '64' '56' '78' '65' '51' '53' '66' '71' '76' '47' '52' '48' '13' '63'
 '68' '15' '57' '70' '73' '80' '67' '5' '84' '94' '86' '3' '69' '6' '17'
 '91' '7' '8' '82' '75' '85' '77' '4' '81' '83' '2' '11' '88' '9' '79'


#### 4.2.1. Cambiar valores con errores y normalizar los datos en función de los valores en el dataset de homicidios:

In [21]:
df_Lesionados['EDAD_VICTIMA'] = df_Lesionados['EDAD_VICTIMA'].replace("sd", "SD")
df_Lesionados['SEXO'] = df_Lesionados['SEXO'].replace("MUJER ", "MUJER")

# Normalizamos valores de "Victima" según los datos del dataset de homicidios:

df_Lesionados.loc[df_Lesionados['VEHICULO_VICTIMA'] == 'CICLISTA', 'VEHICULO_VICTIMA'] = 'BICICLETA'
df_Lesionados.loc[df_Lesionados['VEHICULO_VICTIMA'] == 'TRANSPORTE PUBLICO', 'VEHICULO_VICTIMA'] = 'PASAJEROS'
df_Lesionados.loc[df_Lesionados['VEHICULO_VICTIMA'] == 'CAMIONETA', 'VEHICULO_VICTIMA'] = 'AUTO'
df_Lesionados.loc[df_Lesionados['VEHICULO_VICTIMA'] == 'CAMION', 'VEHICULO_VICTIMA'] = 'CARGAS'
df_Lesionados.loc[df_Lesionados['VEHICULO_VICTIMA'] == 'MONOPATIN', 'VEHICULO_VICTIMA'] = 'OTRO'
df_Lesionados.loc[df_Lesionados['VEHICULO_VICTIMA'] == 'UTILITARIO', 'VEHICULO_VICTIMA'] = 'AUTO'
df_Lesionados.loc[df_Lesionados['VEHICULO_VICTIMA'] == 'TAXI', 'VEHICULO_VICTIMA'] = 'PASAJEROS'
df_Lesionados.loc[df_Lesionados['VEHICULO_VICTIMA'] == 'MIXTO', 'VEHICULO_VICTIMA'] = 'MULTIPLE'

Lo anterior se hace para que al comparar con el datset de homicidios, se cuente con los mismos valores.

In [58]:
print('Victima:',df_Lesionados['VEHICULO_VICTIMA'].unique()) 
print('--------------------------------------------')   
print('sexo:',df_Lesionados['SEXO'].unique())  
print('--------------------------------------------')
print('Edad:',df_Lesionados['EDAD_VICTIMA'].unique()) 
print('--------------------------------------------')
print('Gravedad:',df_Lesionados['GRAVEDAD'].unique())


Victima: ['SD' 'MOTO' 'AUTO' 'TAXI' 'PEATON' 'CICLISTA' 'TRANSPORTE PUBLICO'
 'MOVIL' 'CAMION' 'UTILITARIO' 'MONOPATIN' 'MIXTO' 'OTRO']
--------------------------------------------
sexo: ['VARON' 'SD' 'MUJER' 'MUJER ']
--------------------------------------------
Edad: [57 'SD' 45 27 32 21 37 33 25 23 42 53 1 62 67 75 40 48 3 49 '25' 34 52 44
 28 59 50 73 '26' 51 16 43 38 64 36 84 22 26 '32' 20 5 39 56 69 55 46 29
 41 54 74 70 6 30 12 83 24 35 68 15 31 19 '30' 9 2 13 18 17 58 60 47 '20'
 7 63 66 80 65 71 79 85 61 77 90 '43' '58' '45' 76 '54' '55' 89 '74' '19'
 78 91 97 10 72 88 11 81 '31' 8 14 '50' '21' 96 '27' '34' '40' '28' '59'
 '39' 82 87 86 '38' '37' 'sd' '44' '29' '35' '42' '18' 4 '23' '41' 92 '46'
 '72' '36' '24' '87' '16' '14' '33' '61' '12' '49' '62' '90' '22' '60'
 '64' '56' '78' '65' '51' '53' '66' '71' '76' '47' '52' '48' '13' '63'
 '68' '15' '57' '70' '73' '80' '67' '5' '84' '94' '86' '3' '69' '6' '17'
 '91' '7' '8' '82' '75' '85' '77' '4' '81' '83' '2' '11' '88' '9' '79'


In [59]:
df_Lesionados.columns

Index(['ID hecho', 'AAA', 'MM', 'DD', 'FECHA ', 'VEHICULO_VICTIMA', 'SEXO',
       'EDAD_VICTIMA', 'GRAVEDAD'],
      dtype='object')

#### 4.3. Eliminar columnas innecesarias y cambiar formato de tipos de dato correspondiente:

In [60]:
# Lista de columnas a eliminar
columnas_lec_vic_eliminar = ['AAA', 'MM', 'DD']

# Eliminar las columnas especificadas
df_Lesionados = df_Lesionados.drop(columns=columnas_lec_vic_eliminar)

In [61]:
# Formato de fecha
df_Lesionados['EDAD_VICTIMA'] = pd.to_numeric(df_Lesionados['EDAD_VICTIMA'], errors='coerce')

# Mostrar el DataFrame actualizado
df_Lesionados

,ID hecho,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019-01-01,SD,VARON,57.0,LEVE
1,LC-2019-0000063,2019-01-01,SD,SD,NaN,LEVE
2,LC-2019-0000079,2019-01-01,SD,VARON,NaN,LEVE
3,LC-2019-0000082,2019-01-01,SD,VARON,45.0,LEVE
4,LC-2019-0000082,2019-01-01,SD,MUJER,45.0,LEVE
...,...,...,...,...,...,...
27600,LC-2021-0451911,2021-09-11,TRANSPORTE PUBLICO,VARON,87.0,LEVE
27601,LC-2021-0530228,2021-10-25,TRANSPORTE PUBLICO,MUJER,60.0,LEVE
27602,LC-2021-0530228,2021-10-25,TRANSPORTE PUBLICO,MUJER,32.0,LEVE
27603,LC-2021-0201378,2021-05-02,MOTO,VARON,32.0,LEVE


#### 4.4. Exportar dataframe a CSV en la carpeta "Datos_Limpios"

In [62]:
df_Lesionados.to_csv('Datos_limpios/Victimas_Lesiones_CABA.csv',index=False)